# EJFAT LB Control Plane Tester

This notebook stands up a slice of 3 nodes - sender, receiver and cpnode. The control plane daemon is deployed on the `cpnode` node in a 'mock' configuration (with no FPGA). DAQ and worker node code can be deployed on `sender` and `receiver` nodes for testing. The slice uses 'shared' and is created within a single FABRIC site for simplicity. It uses a single L2 bridge connection with RFC1918 IPv4 addressing, allowing all nodes to talk to each other. It is possible to run the dataplane assuming a single worker can keep up with a single sender since no actual load balancer is present in this configuration.

Slice example:

<div>
    <img src="figs/UDP LB Control Plane Testing slice.png" width=500>
</div>

## Preparation and overview

- Be sure to [generate a keypair for Jupyter Hub](GitHubSSH.ipynb) and register it with GitHub - the keys will be used to check out the code from private repositories, like [UDPLBd](https://github.com/esnet/udplbd) and [E2SAR](https://github.com/JeffersonLab/E2SAR).
- Note that for E2SAR development and testing sender and receiver node compile/build environments will be setup via post-boot scripts ([sender](post-boot/sender.sh) and [receiver](post-boot/recver.sh)) and grpc++ is installed as a tar.gz with static and dynamic libraries compiled for ubuntu22
- This does not setup the control plane node for anything, but testing a specific version - you can set which branch of UDPLBd to check out and a containerized version is built and stood up.

## Preamble

This cell must be executed whether you are creating a new slice or continuing work on the old one. If you are continuing work, you then skip the slice create section and proceed to wherever you left off.

In [1]:
#
# EDIT THIS
#
# if you want to force a site instead of using random
site_override = 'UCSD'
#site_override = None

# GitHub SSH key file (private) registered using the GitHubSSH.ipynb notebook referenced above
github_key = '/home/fabric/work/fabric_config/github_ecdsa'
# note that the below is distribution specific ('ubuntu' for ubuntu and so on)
vm_key_location = '/home/ubuntu/.ssh/github_ecdsa'

# branches for UDPLBd and E2SAR that we want checked out on the VMs
udplbd_branch = 'main'
e2sar_branch = 'api-skeleton'

# grpc tar ball stored in E2SAR repo (via LFS)
grpc_tar = "grpc-v1.54.1-ubuntu22.tar.gz"

#
# SHOULDN'T NEED TO EDIT BELOW
#
# Preamble
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# variable settings
slice_name = f'UDP LB Control Plane Testing with udplbd[{udplbd_branch}], e2sar[{e2sar_branch}]'
# for each node specify IP address (assuming /24), OS image
# note that most of the keys in these dictionaries map directly
# onto parameters to add_node()
node_config = {
    'sender': {
        'ip':'192.168.0.1', 
        'image':'default_ubuntu_22',
        'cores': 8,
        'ram': 8,
        'disk': 100 },
    'recver': {
        'ip':'192.168.0.2', 
        'image':'default_ubuntu_22',
        'cores':8,
        'ram': 8,
        'disk': 100 },
    'cpnode': {
        'ip':'192.168.0.3', 
        'image':'docker_ubuntu_22',
        'cores':8,
        'ram': 8,
        'disk': 100 },
}
# skip these keys as they are not part of add_node params
skip_keys = ['ip']
# this is the NIC to use
nic_model = 'NIC_Basic'
# the subnet should match IPs
subnet = IPv4Network("192.168.1.0/24")

def execute_commands(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,baldin_0000147304
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the slice

In [2]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

ID,Name,Lease Expiration (UTC),Lease Start (UTC),Project ID,State
56b92d26-0926-4a62-9061-04cafa467fec,UDP LB Control Plane Testing slice,2024-05-22 17:19:02 +0000,2024-05-21 17:19:04 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK


In [3]:
# List available images (this step is optional)
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

Available images are: ['default_centos8_stream', 'default_centos9_stream', 'default_centos_7', 'default_centos_8', 'default_debian_10', 'default_debian_11', 'default_fedora_35', 'default_rocky_8', 'default_rocky_9', 'default_ubuntu_18', 'default_ubuntu_20', 'default_ubuntu_21', 'default_ubuntu_22', 'default_fedora_36', 'default_fedora_37', 'docker_rocky_8', 'docker_ubuntu_20', 'docker_ubuntu_22']


In [4]:
# find an available site in continental US
lon_west=-124.3993243
lon_east=-69.9721573

# getting a random site make take a bit of time
if not site_override:
    selected_site = fablib.get_random_site(filter_function=lambda x: x['location'][1] < lon_east
                                              and x['location'][1] > lon_west) 
else:
    selected_site = site_override

if selected_site:
    print(f'Selected site is {selected_site}')
else:
    print('Unable to find a site matching the requirements')

# write selected site into node attributes
for n in node_config:
    node_config[n]['site'] = selected_site
    

Selected site is UCSD


In [5]:
# build a slice
slice = fablib.new_slice(name=slice_name)

# create a network
net1 = slice.add_l2network(name='site_bridge_net', subnet=subnet)

nodes = dict()
# create  nodes for sending and receiving with a selected network card
# use subnet address assignment
for node_name, node_attribs in node_config.items():
    print(f"{node_name=} {node_attribs['ip']}")
    nodes[node_name] = slice.add_node(name=node_name, **{x: node_attribs[x] for x in node_attribs if x not in skip_keys})
    nic_interface = nodes[node_name].add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_attribs['ip'])
    # postboot configuration is under 'post-boot' directory
    nodes[node_name].add_post_boot_upload_directory('post-boot','.')
    nodes[node_name].add_post_boot_execute(f'chmod +x post-boot/{node_name}.sh && ./post-boot/{node_name}.sh')

# Submit the slice
slice.submit();


Retry: 8, Time: 355 sec


ID,1e5f0974-7de0-4ce6-9800-6535e8a03394
Name,"UDP LB Control Plane Testing with udplbd[main], e2sar[api-skeleton]"
Lease Expiration (UTC),2024-05-23 14:25:08 +0000
Lease Start (UTC),2024-05-22 14:25:10 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
86295be4-78a4-4d84-8dc3-9e8e2be1bf99,cpnode,8,8,100,docker_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.190,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.190,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0a1755bc-3165-485d-8b8f-1e34ce9e8ca1,recver,8,8,100,default_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.172,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.172,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dbf7e4ee-06f9-4032-a027-db24ee17b2d1,sender,8,8,100,default_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.168,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.168,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
59f1ed75-45fa-42fc-b923-5e27e1c3da1e,site_bridge_net,L2,L2Bridge,UCSD,192.168.1.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
sender-sender_NIC_Basic_nic-p1,p1,sender,site_bridge_net,100,config,,06:9F:D6:8E:7E:58,enp6s0,enp6s0,192.168.0.1,4
recver-recver_NIC_Basic_nic-p1,p1,recver,site_bridge_net,100,config,,06:AE:3B:35:9F:83,enp6s0,enp6s0,192.168.0.2,4
cpnode-cpnode_NIC_Basic_nic-p1,p1,cpnode,site_bridge_net,100,config,,06:CE:FC:94:3E:55,enp6s0,enp6s0,192.168.0.3,4



Time to print interfaces 360 seconds


## Get Slice Details

If not creating a new slice, and just continuing work on an existing one, execute this cell (in addition to the preamble) and then any of the cells below will work.

In [6]:
# get slice details (if not creating new)
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

cpnode = slice.get_node(name="cpnode")    
sender = slice.get_node(name="sender")
recver = slice.get_node(name="recver")

ID,1e5f0974-7de0-4ce6-9800-6535e8a03394
Name,"UDP LB Control Plane Testing with udplbd[main], e2sar[api-skeleton]"
Lease Expiration (UTC),2024-05-23 14:25:08 +0000
Lease Start (UTC),2024-05-22 14:25:10 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
59f1ed75-45fa-42fc-b923-5e27e1c3da1e,site_bridge_net,L2,L2Bridge,UCSD,192.168.1.0/24,None,Active,


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
86295be4-78a4-4d84-8dc3-9e8e2be1bf99,cpnode,8,8,100,docker_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.190,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.190,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0a1755bc-3165-485d-8b8f-1e34ce9e8ca1,recver,8,8,100,default_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.172,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.172,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dbf7e4ee-06f9-4032-a027-db24ee17b2d1,sender,8,8,100,default_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.168,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.168,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## Start the UDPLBd container

In [ ]:
# check if any dockers are running already and that we have compose and buildx installed by post-boot script
commands = [
    'docker container ls',
    'docker compose version',
    'docker buildx version'
]
execute_commands(cpnode, commands)

In [ ]:
# upload the mock config file for UDPLBd
result = cpnode.upload_file('config/lb_mock.yml','lb_mock.yml')

# upload the GitHub SSH key onto the VM
result = cpnode.upload_file(github_key, vm_key_location)

# checkout UDPLBd (including the right branch) using that key
commands = [
    f"chmod go-rwx {vm_key_location}",
    f"GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone -b {udplbd_branch} git@github.com:esnet/udplbd.git",
]

execute_commands(cpnode, commands)

In [ ]:
# start the UDPLBd container
commands = [
    f'cp lb_mock.yml ./udplbd/etc/config.yml',
    f'cd udplbd; docker compose up -d'
]

execute_commands(cpnode, commands)

In [ ]:
# check the logs
commands = [
    'docker compose ls',
    'cd udplbd; docker compose logs'
]

execute_commands(cpnode, commands)

## Clone E2SAR code into sender and receiver

In [7]:
# install github ssh key and set up build environment variables for interactive logins
commands = [
    f"chmod go-rwx {vm_key_location}",
    f"echo 'export PKG_CONFIG_PATH=/home/ubuntu/grpc-install/lib/pkgconfig/ PATH=/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=/home/ubuntu/grpc-install/lib/' >> ~/.profile",
]

for node in [sender, recver]:    
    # upload the GitHub SSH key onto the VM
    result = node.upload_file(github_key, vm_key_location)
    execute_commands(node, commands)

	Executing "chmod go-rwx /home/ubuntu/.ssh/github_ecdsa" on node sender
	Executing "echo 'export PKG_CONFIG_PATH=/home/ubuntu/grpc-install/lib/pkgconfig/ PATH=/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=/home/ubuntu/grpc-install/lib/' >> ~/.profile" on node sender
	Executing "chmod go-rwx /home/ubuntu/.ssh/github_ecdsa" on node recver
	Executing "echo 'export PKG_CONFIG_PATH=/home/ubuntu/grpc-install/lib/pkgconfig/ PATH=/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=/home/ubuntu/grpc-install/lib/' >> ~/.profile" on node recver


In [8]:
# checkout E2SAR (including the right branch) using that key, install grpc binary that is stored in the repo
commands = [
    f"GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b {e2sar_branch} git@github.com:JeffersonLab/E2SAR.git",
    f"tar -zxf E2SAR/binary_artifacts/{grpc_tar}",
    #f"cd E2SAR; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git submodule init",
]

for node in [sender, recver]:    
    execute_commands(node, commands)

	Executing "GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b api-skeleton git@github.com:JeffersonLab/E2SAR.git" on node sender
Submodule path 'udplbd': checked out 'ba7e144e900ca66253d928617b428b0f2f5c9316'
 Cloning into 'E2SAR'...
Submodule 'udplbd' (git@github.com:/esnet/udplbd.git) registered for path 'udplbd'
Cloning into '/home/ubuntu/E2SAR/udplbd'...
 	Executing "tar -zxf E2SAR/binary_artifacts/grpc-v1.54.1-ubuntu22.tar.gz" on node sender
	Executing "GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b api-skeleton git@github.com:JeffersonLab/E2SAR.git" on node recver
Submodule path 'udplbd': checked out 'ba7e144e900ca66253d928617b428b0f2f5c9316'
 Cloning into 'E2SAR'...
Submodule 'udplbd' (git@github.com:/esnet/udplbd.gi

In [9]:
# build E2SAR code
commands = [
    f"cd E2SAR; PKG_CONFIG_PATH=/home/ubuntu/grpc-install/lib/pkgconfig/ PATH=/home/ubuntu/.local/bin:/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=/home/ubuntu/grpc-install/lib/ meson setup builddir",
    f"cd E2SAR/builddir; PKG_CONFIG_PATH=/home/ubuntu/grpc-install/lib/pkgconfig/ PATH=/home/ubuntu/.local/bin:/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=/home/ubuntu/grpc-install/lib/  meson compile"
]

for node in [sender, recver]:    
    execute_commands(node, commands)

	Executing "cd E2SAR; PKG_CONFIG_PATH=/home/ubuntu/grpc-install/lib/pkgconfig/ PATH=/home/ubuntu/.local/bin:/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=/home/ubuntu/grpc-install/lib/ meson setup builddir" on node sender
The Meson build system
Version: 1.4.0
Source dir: /home/ubuntu/E2SAR
Build dir: /home/ubuntu/E2SAR/builddir
Build type: native build
Project name: E2SAR library
Project version: 0.1
C++ compiler for the host machine: c++ (gcc 11.4.0 "c++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0")
C++ linker for the host machine: c++ ld.bfd 2.38
Host machine cpu family: x86_64
Host machine cpu: x86_64
Found pkg-config: YES (/usr/bin/pkg-config) 0.29.2
Run-time dependency glib-2.0 found: YES 2.72.4
Run-time dependency grpc++ found: YES 1.54.1
Run-time dependency protobuf found: YES 3.21.12.0
Run-time dependency threads found: YES
Run-time dependency Boost (found: system) found: YES 1.74.0 (/usr)
Dependency grpc++ found: YES 1.54.1 (cached)
Program protoc found: YES (/home/ubuntu/grpc

## Manage the slice

### Extend

In [ ]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Delete

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()